<a href="https://colab.research.google.com/github/how-to-train-your-model/data-science-portfolio/blob/main/451_A8_Finding_important_clusters_of_predictive_economic_factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 5 Coding Assignment: 
Finding important clusters of predictive
economic factors of SPX index


# Libraries

In [1]:
!pip install sweetviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 80.8 MB/s eta 0:00:00


In [2]:
# load libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import sweetviz as sv

from tqdm import tqdm
from sklearn.cluster import KMeans, MiniBatchKMeans, SpectralClustering, AgglomerativeClustering
from sklearn.datasets import make_regression, make_classification, make_blobs
from sklearn.metrics import silhouette_samples, log_loss, accuracy_score, r2_score, roc_auc_score, f1_score,mean_squared_error, mean_absolute_error, adjusted_rand_score
from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import load_boston, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection._split import KFold




In [3]:
# Allows for muliple output in one cell window
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Allows for viewing the complete data in a single window 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ls -l "/content/drive/My Drive/Colab_Notebooks/MSDS-451/W8"

Mounted at /content/drive
total 852
-rw------- 1 root root  37151 Mar 12 23:07 '451_A8_Finding important clusters of predictive economic factors.ipynb'
-rw------- 1 root root 249811 Mar 11 03:27 'Copy of MSDS451_A5_Pant_Coding Assignment: MDA on economic factors as predictors of S&P 500 index.docx'
-rw------- 1 root root 301093 Mar 12 00:18 'MSDS 451 Assignment_07_Chan-1.ipynb'
-rw------- 1 root root 244386 Feb 19 18:18  S_P_mnthly_returns_input-3.csv
-rw------- 1 root root  13711 Feb 19 18:18  S_P_mnthly_returns_output-3.csv
-rw------- 1 root root  24148 Mar  4 18:50  SPY.csv


In [5]:
w8_files = "/content/drive/My Drive/Colab_Notebooks/MSDS-451/W8"

X = pd.read_csv(w8_files+"/S_P_mnthly_returns_input-3.csv")
Y = pd.read_csv(w8_files+"/S_P_mnthly_returns_output-3.csv")

X.set_index('yyyymm', inplace=True)
Y.set_index('yyyymm', inplace=True)
#y = Y.iloc[:,0] 
# why so float64 ????????
#y = y.astype('float64')

print(X.shape)
X.head()


(888, 15)


,ntis,infl,ltr,corpr,svar,d/p,d/y,e/p,d/e,tms,dfy,dfr,b/m_FD,tbl_FD,lty_FD
yyyymm,,,,,,,,,,,,,,,
1945-12-01,0.025525,0.005525,0.0194,0.0133,0.001538,-3.269684,-3.337063,-2.894991,-0.374693,0.0161,0.0049,-0.0061,-0.216009,-2.99635,-1.507835
1946-01-01,0.027969,0.000000,0.0025,0.0128,0.002645,-3.327012,-3.255014,-2.983423,-0.343589,0.0161,0.0047,0.0103,-0.264780,-2.99635,-1.486065
1946-02-01,0.031091,-0.005495,0.0032,0.0034,0.005942,-3.245065,-3.290322,-2.932931,-0.312134,0.0160,0.0047,0.0002,-0.167204,-2.99635,-1.481915
1946-03-01,0.031188,0.011050,0.0010,0.0034,0.001497,-3.280469,-3.317389,-3.000167,-0.280302,0.0160,0.0047,0.0024,-0.190201,-2.99635,-1.474585
1946-04-01,0.031307,0.005464,-0.0135,-0.0043,0.001078,-3.317389,-3.339531,-3.059560,-0.257829,0.0169,0.0050,0.0092,-0.222940,-2.99635,-1.425691


In [6]:
print(Y.shape)
Y.head()

(888, 1)


,ret
yyyymm,
1945-12-01,1.0
1946-01-01,-1.0
1946-02-01,1.0
1946-03-01,1.0
1946-04-01,1.0


In [7]:
SPY_data = pd.read_csv(w8_files+ "/SPY.csv")
SPY_data.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-02-01,43.96875,45.12500,42.81250,44.40625,26.160336,5417600
1,1993-03-01,44.56250,45.84375,44.21875,45.18750,26.620569,3019200


# Functions (from Scott)

In [8]:

#---------------------------------------------------
def clusterKMeansBase_0(corr0,maxNumClusters,n_init,seed):
    x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
    for init in range(1,n_init):
        for i in range(2,maxNumClusters+1):
            kmeans_=KMeans(n_clusters=i,n_jobs=1,n_init=n_init,random_state=seed)
            kmeans_=kmeans_.fit(x)
            silh_=silhouette_samples(x,kmeans_.labels_)
            stat=(silh_.mean()/silh_.std(),silh.mean()/silh.std())
            if np.isnan(stat[1]) or stat[0]>stat[1]:
                silh,kmeans=silh_,kmeans_
    newIdx=np.argsort(kmeans.labels_)
    corr1=corr0.iloc[newIdx] # reorder rows
    corr1=corr1.iloc[:,newIdx] # reorder columns
    clstrs={i:corr0.columns[np.where(kmeans.labels_==i)[0]].tolist() \
        for i in np.unique(kmeans.labels_) } # cluster members
    silh=pd.Series(silh,index=x.index)
    return corr1,clstrs,silh
#---------------------------------------------------
def clusterKMeansBase(corr0):
    x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix 
    #x,silh=(1-(corr0.fillna(0))**2)**.5,pd.Series()# observations matrix 
    #kmeans_=AgglomerativeClustering(affinity='precomputed',linkage='average')
    maxNumClusters = int(corr0.shape[1]/5)
    for i in range(2,maxNumClusters+1):
        kmeans_=AgglomerativeClustering(n_clusters=i, affinity='precomputed',linkage='average')
        kmeans=kmeans_.fit(x)
        silh_=silhouette_samples(x,kmeans_.labels_)
        stat=(silh_.mean()/silh_.std(),silh.mean()/silh.std())
        if np.isnan(stat[1]) or stat[0]>stat[1]:
            silh,kmeans=silh_,kmeans_
    newIdx=np.argsort(kmeans.labels_)
    corr1=corr0.iloc[newIdx] # reorder rows
    corr1=corr1.iloc[:,newIdx] # reorder columns
    clstrs={i:corr0.columns[np.where(kmeans.labels_==i)[0]].tolist() \
        for i in np.unique(kmeans.labels_) } # cluster members
    silh=pd.Series(silh,index=x.index)
    return corr1,clstrs,silh
#---------------------------------------------------
def clusterKMeansTop_0(corr0):
    corr1,clstrs,silh=clusterKMeansBase(corr0)
    clusterTstats={i:np.mean(silh[clstrs[i]])/ \
        np.std(silh[clstrs[i]]) for i in clstrs.keys() if len(clstrs[i])>1}
    tStatMean=sum(np.array(list(clusterTstats.values())))/len(clusterTstats)
    redoClusters=[i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    if len(redoClusters)<=1:
        return corr1,clstrs,silh
    else:
        keysRedo=[j for i in redoClusters for j in clstrs[i]]
        corrTmp=corr0.loc[keysRedo,keysRedo]
        tStatMean=np.mean([clusterTstats[i] for i in redoClusters])
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp)
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0, \
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, \
            clstrs2)
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys() if len(clstrsNew[i])>1])
        if newTstatMean<=tStatMean:
            return corr1,clstrs,silh
        else:
            return corrNew,clstrsNew,silhNew
#---------------------------------------------------
def makeNewOutputs(corr0,clstrs,clstrs2):
    clstrsNew={}
    for i in clstrs.keys():
        clstrsNew[len(clstrsNew.keys())]=list(clstrs[i])
    for i in clstrs2.keys():
        clstrsNew[len(clstrsNew.keys())]=list(clstrs2[i])
    newIdx=[j for i in clstrsNew for j in clstrsNew[i]]
    corrNew=corr0.loc[newIdx,newIdx]
    x=((1-corr0.fillna(0))/2.)**.5
    kmeans_labels=np.zeros(len(x.columns))
    for i in clstrsNew.keys():
        idxs=[x.index.get_loc(k) for k in clstrsNew[i]]
        kmeans_labels[idxs]=i
    silhNew=pd.Series(silhouette_samples(x,kmeans_labels),index=x.index)
    return corrNew,clstrsNew,silhNew
#---------------------------------------------------
def clusterKMeansTop_0(corr0,maxNumClusters,n_init):
    if maxNumClusters==None:maxNumClusters=corr0.shape[1]-1
    corr1,clstrs,silh=clusterKMeansBase_0(corr0,maxNumClusters= \
        min(maxNumClusters,corr0.shape[1]-1),n_init=n_init)
    clusterTstats={i:np.mean(silh[clstrs[i]])/ \
        np.std(silh[clstrs[i]]) for i in clstrs.keys()}
    tStatMean=sum(clusterTstats.values())/len(clusterTstats)
    redoClusters=[i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    if len(redoClusters)<=1:
        return corr1,clstrs,silh
    else:
        keysRedo=[j for i in redoClusters for j in clstrs[i]]
        corrTmp=corr0.loc[keysRedo,keysRedo]
        tStatMean=np.mean([clusterTstats[i] for i in redoClusters])
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp, \
            maxNumClusters=min(maxNumClusters, \
            corrTmp.shape[1]-1),n_init=n_init)
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0, \
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, \
            clstrs2)
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys()])
        if newTstatMean<=tStatMean:
            return corr1,clstrs,silh
        else:
            return corrNew,clstrsNew,silhNew
#---------------------------------------------------
def clusterKMeansTop(corr0):
    corr1,clstrs,silh=clusterKMeansBase(corr0)
    clusterTstats={i:np.mean(silh[clstrs[i]])/ \
        np.std(silh[clstrs[i]]) for i in clstrs.keys() if len(clstrs[i])>1}
    tStatMean=sum(np.array(list(clusterTstats.values())))/len(clusterTstats)
    redoClusters=[i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    if len(redoClusters)<=1:
        return corr1,clstrs,silh
    else:
        keysRedo=[j for i in redoClusters for j in clstrs[i]]
        corrTmp=corr0.loc[keysRedo,keysRedo]
        tStatMean=np.mean([clusterTstats[i] for i in redoClusters])
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp)
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0, \
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, \
            clstrs2)
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys() if len(clstrsNew[i])>1])
        if newTstatMean<=tStatMean:
            return corr1,clstrs,silh
        else:
            return corrNew,clstrsNew,silhNew
#---------------------------------------------------
def groupMeanStd(df0,clstrs):
    out=pd.DataFrame(columns=['mean','std'])
    for i,j in clstrs.iteritems():
        df1=df0[j].sum(axis=1)
        out.loc['C_'+str(i),'mean']=df1.mean()
        out.loc['C_'+str(i),'std']=df1.std()*df1.shape[0]**-.5
    return out
#------------------------------------------------------
def featImpMDA_Clustered_ldp(clf,X,y,clstrs,n_splits=10):
    cvGen=KFold(n_splits=n_splits)
    scr0,scr1=pd.Series(),pd.DataFrame(columns=clstrs.keys())
    for i,(train,test) in enumerate(cvGen.split(X=X)):
        X0,y0=X.iloc[train,:],y[train]
        X1,y1=X.iloc[test,:],y[test]
        fit=clf.fit(X=X0,y=y0)
        prob=fit.predict_proba(X1)
        scr0.loc[i]=-log_loss(y1,prob,labels=clf.classes_)
        for j in scr1.columns:
            X1_=X1.copy(deep=True)
            for k in clstrs[j]:
                np.random.shuffle(X1_[k].values) # shuffle cluster
            prob=fit.predict_proba(X1_)
            scr1.loc[i,j]=-log_loss(y1,prob,labels=clf.classes_)
    imp=(-1*scr1).add(scr0,axis=0)
    imp=imp/(-1*scr1)
    imp=pd.concat({'mean':imp.mean(),'std':imp.std()*imp.shape[0]**-.5},axis=1)
    imp.index=['C_'+str(i) for i in imp.index]
    return imp
#------------------------------------------------------
def featImpMDA_Clustered(clf,X_valid,y_valid,clstrs,n_iterations=100):
    mda_df = []
    n_class = len(set(y_valid))
    if n_class < 2:
        raise ValueError(f'The number of classes for target variable is {n_class}')
    for i in tqdm(range(n_iterations)):
        mda_scores = {}
        if n_class == 2:
            acc = roc_auc_score(y_valid, clf.predict_proba(X_valid)[:,1])
        elif n_class > 2:
            acc = r2_score(y_valid, clf.predict(X_valid))
        for j in clstrs.keys():
            X_t = X_valid.copy()
            for k in clstrs[j]:
                np.random.shuffle(X_t[k].values)
            if n_class == 2:
                shuff_acc = roc_auc_score(y_valid, clf.predict_proba(X_t)[:,1])
            elif n_class > 2:
                shuff_acc = r2_score(y_valid, clf.predict(X_t))
            mda_scores[f'C_{j}'] = (acc-shuff_acc)/acc
        mda_df.append(pd.DataFrame(mda_scores,index=[0]).T)
    mda_df = pd.concat(mda_df,axis=1).T.reset_index(drop=True)
    imp_sorted = new_score(mda_df,mda_df.shape[1])[1]
    top_clusters = [int(c.split('_')[1]) for c in imp_sorted[imp_sorted>=np.mean(imp_sorted)].index]
    selected_features = []
    for indx in top_clusters:
        selected_features += clstrs[indx]
    return mda_df, selected_features, top_clusters, imp_sorted
#-----------------------------------------------------
def new_score(df,m0):
    rank_df = df.copy()
    for i in range(rank_df.shape[0]):
        rank_df.iloc[i] = rank_df.iloc[i].rank(ascending=False,method='min')
    top_features = rank_df.mean().sort_values()
    top_scores = (1/top_features).head(int(m0))/sum(1/top_features)
    return sum(top_scores), top_scores
def new_criterion(df,m0):
    rank_df = df.copy()
    for i in range(rank_df.shape[0]):
        rank_df.iloc[i] = rank_df.iloc[i].rank(ascending=False,method='min')
    top_features = rank_df.mean().sort_values().head(int(m0))
    return np.sqrt(rank_df[top_features.index].var().mean()), top_features, rank_df[top_features.index]

# EDA

In [9]:
df_EDA = X.copy()

In [10]:
print("\n\n Duplicates",df_EDA.duplicated().sum())
#my_report = sv.analyze(df_EDA)
#my_report.show_html()



 Duplicates 0


In [11]:
print("\n\n Missing Values before dropping/Imputing \n",df_EDA.isna().sum())



 Missing Values before dropping/Imputing 
 ntis      0
infl      0
ltr       0
corpr     0
svar      0
d/p       0
d/y       0
e/p       0
d/e       0
tms       0
dfy       0
dfr       0
b/m_FD    0
tbl_FD    0
lty_FD    0
dtype: int64


# Question 1

Set the random seed to 1. Apply the distance metric based on correlations to the features used in Modules 5-7 in predicting the sign of SPX monthly returns, and apply K-means clustering to those features, using an optimal number of clusters as defined in MLAM section 4.4.2. Use Snippet 4.1 in MLAM. How many clusters did you find? What are the top 2 clusters? (2 pt)

In [12]:
start_date = '2012-06-01'
end_date = '2019-10-01'

In [13]:
import datetime
SPY_data['Date']= pd.to_datetime(SPY_data['Date'])
# Lets filter out the returns file with the date range we will be using for test data.
SPY_data_test = SPY_data.loc[( SPY_data['Date'] >= datetime.datetime(2012, 6, 1) )]
SPY_data_test = SPY_data_test.loc[( SPY_data['Date'] <= datetime.datetime(2019, 10, 1) )]

In [14]:
SPY_data_test.set_index('Date', inplace=True)
SPY_data_test.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-06-01,129.410004,136.270004,127.139999,136.100006,114.001549,3644886200
2012-07-01,136.479996,139.339996,132.600006,137.710007,115.947792,2864743300
2012-08-01,138.699997,143.089996,135.580002,141.160004,118.852539,2413590900
2012-09-01,141.039993,148.110001,140.130005,143.970001,121.218559,2391233500
2012-10-01,144.520004,147.160004,140.389999,141.350006,119.647865,2719915500


In [15]:
np.random.seed(1)
x_train = X[:start_date]
x_test = X[start_date:]
y_train = Y[:start_date]
y_test = Y[start_date:]

In [16]:
x_train.shape
y_train.shape

(799, 15)

(799, 1)

In [17]:
#---------------------------------------------------
def clusterKMeansBase_0(corr0,maxNumClusters,n_init,seed):
    x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
    for init in range(1,n_init):
        for i in range(2,maxNumClusters+1):
            kmeans_=KMeans(n_clusters=i,
                           #n_jobs=1,
                           max_iter = 10,
                           n_init=n_init,random_state=seed)
            kmeans_=kmeans_.fit(x)
            silh_=silhouette_samples(x,kmeans_.labels_)
            stat=(silh_.mean()/silh_.std(),silh.mean()/silh.std())
            if np.isnan(stat[1]) or stat[0]>stat[1]:
                silh,kmeans=silh_,kmeans_
    newIdx=np.argsort(kmeans.labels_)
    corr1=corr0.iloc[newIdx] # reorder rows
    corr1=corr1.iloc[:,newIdx] # reorder columns
    clstrs={i:corr0.columns[np.where(kmeans.labels_==i)[0]].tolist() \
        for i in np.unique(kmeans.labels_) } # cluster members
    silh=pd.Series(silh,index=x.index)
    return corr1,clstrs,silh

In [18]:
#clusterKMeansBase_0(corr0,maxNumClusters,n_init,seed)
corr1a,clstrs1a,silh1a = clusterKMeansBase_0(x_train.corr(),
                                        maxNumClusters = 10,
                                        n_init = 7, 
                                        seed = 1)

<ipython-input-17-fb9cb799cfd1>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix


In [19]:
corr1a,clstrs1a,silh1a

(            ntis      infl       ltr     corpr      svar       d/e       tms       dfy       dfr    tbl_FD    lty_FD       d/p       d/y       e/p    b/m_FD
 ntis    1.000000  0.103672 -0.109347 -0.100199 -0.240883  0.065750 -0.136656 -0.419595  0.039599  0.046816 -0.171648  0.224010  0.226061  0.171699  0.262926
 infl    0.103672  1.000000 -0.114806 -0.133403 -0.101489 -0.066881 -0.168796  0.030914 -0.008107  0.171720  0.204226  0.168510  0.177266  0.204729  0.249961
 ltr    -0.109347 -0.114806  1.000000  0.863032  0.140054 -0.033982  0.011898  0.148486 -0.456060 -0.038341 -0.079555 -0.027153 -0.036133 -0.003916 -0.020313
 corpr  -0.100199 -0.133403  0.863032  1.000000  0.080928  0.034088  0.061700  0.206720  0.055963 -0.066987 -0.058566 -0.021229 -0.033736 -0.042460 -0.035616
 svar   -0.240883 -0.101489  0.140054  0.080928  1.000000  0.154275  0.150383  0.309197 -0.134243 -0.205400 -0.021984 -0.088276 -0.077671 -0.184699 -0.059802
 d/e     0.065750 -0.066881 -0.033982  0.034088  0.1

# Question 2

Set the random seed to 2. Repeat Part 1. Did the number of clusters change from what you found in Part 1? Did the top 2 clusters change? (I.e. Do they have identical constituents?) (2 pt)

In [20]:
#clusterKMeansBase_0(corr0,maxNumClusters,n_init,seed)
corr2,clstrs2,silh2 = clusterKMeansBase_0(x_train.corr(),
                                        maxNumClusters = 10,
                                        n_init = 5, 
                                        seed = 2)

<ipython-input-17-fb9cb799cfd1>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix


In [21]:
corr2,clstrs2,silh2

(            ntis      infl       ltr     corpr      svar       d/e       tms       dfy       dfr    tbl_FD    lty_FD       d/p       d/y       e/p    b/m_FD
 ntis    1.000000  0.103672 -0.109347 -0.100199 -0.240883  0.065750 -0.136656 -0.419595  0.039599  0.046816 -0.171648  0.224010  0.226061  0.171699  0.262926
 infl    0.103672  1.000000 -0.114806 -0.133403 -0.101489 -0.066881 -0.168796  0.030914 -0.008107  0.171720  0.204226  0.168510  0.177266  0.204729  0.249961
 ltr    -0.109347 -0.114806  1.000000  0.863032  0.140054 -0.033982  0.011898  0.148486 -0.456060 -0.038341 -0.079555 -0.027153 -0.036133 -0.003916 -0.020313
 corpr  -0.100199 -0.133403  0.863032  1.000000  0.080928  0.034088  0.061700  0.206720  0.055963 -0.066987 -0.058566 -0.021229 -0.033736 -0.042460 -0.035616
 svar   -0.240883 -0.101489  0.140054  0.080928  1.000000  0.154275  0.150383  0.309197 -0.134243 -0.205400 -0.021984 -0.088276 -0.077671 -0.184699 -0.059802
 d/e     0.065750 -0.066881 -0.033982  0.034088  0.1

# Q1&2: Complete data Train and Test

In [22]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(X, Y, test_size=0.30)
                                                        #,random_state = 1)

#clusterKMeansBase_0(corr0,maxNumClusters,n_init,seed)
corr1,clstrs1,silh1 = clusterKMeansBase_0(x_train.corr(),
                                        maxNumClusters = 10,
                                        n_init = 7, 
                                        seed = 1)
corr1,clstrs1,silh1

<ipython-input-17-fb9cb799cfd1>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix


(            ntis      infl       ltr     corpr      svar       d/e       tms       dfy       dfr    tbl_FD    lty_FD       d/p       d/y       e/p    b/m_FD
 ntis    1.000000  0.103672 -0.109347 -0.100199 -0.240883  0.065750 -0.136656 -0.419595  0.039599  0.046816 -0.171648  0.224010  0.226061  0.171699  0.262926
 infl    0.103672  1.000000 -0.114806 -0.133403 -0.101489 -0.066881 -0.168796  0.030914 -0.008107  0.171720  0.204226  0.168510  0.177266  0.204729  0.249961
 ltr    -0.109347 -0.114806  1.000000  0.863032  0.140054 -0.033982  0.011898  0.148486 -0.456060 -0.038341 -0.079555 -0.027153 -0.036133 -0.003916 -0.020313
 corpr  -0.100199 -0.133403  0.863032  1.000000  0.080928  0.034088  0.061700  0.206720  0.055963 -0.066987 -0.058566 -0.021229 -0.033736 -0.042460 -0.035616
 svar   -0.240883 -0.101489  0.140054  0.080928  1.000000  0.154275  0.150383  0.309197 -0.134243 -0.205400 -0.021984 -0.088276 -0.077671 -0.184699 -0.059802
 d/e     0.065750 -0.066881 -0.033982  0.034088  0.1

In [23]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.30)
                                                        #,random_state = 2)

#clusterKMeansBase_0(corr0,maxNumClusters,n_init,seed)
corr2,clstrs2,silh2 = clusterKMeansBase_0(x_train.corr(),
                                        maxNumClusters = 10,
                                        n_init = 7, 
                                        seed = 2)
corr2,clstrs2,silh2

<ipython-input-17-fb9cb799cfd1>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix


(            ntis      infl       ltr     corpr      svar       d/e       tms       dfy       dfr    tbl_FD    lty_FD       d/p       d/y       e/p    b/m_FD
 ntis    1.000000  0.103672 -0.109347 -0.100199 -0.240883  0.065750 -0.136656 -0.419595  0.039599  0.046816 -0.171648  0.224010  0.226061  0.171699  0.262926
 infl    0.103672  1.000000 -0.114806 -0.133403 -0.101489 -0.066881 -0.168796  0.030914 -0.008107  0.171720  0.204226  0.168510  0.177266  0.204729  0.249961
 ltr    -0.109347 -0.114806  1.000000  0.863032  0.140054 -0.033982  0.011898  0.148486 -0.456060 -0.038341 -0.079555 -0.027153 -0.036133 -0.003916 -0.020313
 corpr  -0.100199 -0.133403  0.863032  1.000000  0.080928  0.034088  0.061700  0.206720  0.055963 -0.066987 -0.058566 -0.021229 -0.033736 -0.042460 -0.035616
 svar   -0.240883 -0.101489  0.140054  0.080928  1.000000  0.154275  0.150383  0.309197 -0.134243 -0.205400 -0.021984 -0.088276 -0.077671 -0.184699 -0.059802
 d/e     0.065750 -0.066881 -0.033982  0.034088  0.1

In [24]:
x_train42, x_test42, y_train42, y_test42 = train_test_split(X, Y, test_size=0.3,
                                                        random_state = 42)

#clusterKMeansBase_0(corr0,maxNumClusters,n_init,seed)
corr42,clstrs42,silh42 = clusterKMeansBase_0(x_train.corr(),
                                        maxNumClusters = 10,
                                        n_init = 7, 
                                        seed = 42)
corr42,clstrs42,silh42

<ipython-input-17-fb9cb799cfd1>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix


(             d/p       d/y       e/p    b/m_FD      ntis      infl       ltr     corpr      svar       d/e       tms       dfy       dfr    tbl_FD    lty_FD
 d/p     1.000000  0.995192  0.780559  0.876660  0.224010  0.168510 -0.027153 -0.021229 -0.088276  0.271839 -0.233105  0.094103  0.016267  0.123834 -0.071816
 d/y     0.995192  1.000000  0.776258  0.877039  0.226061  0.177266 -0.036133 -0.033736 -0.077671  0.271377 -0.237776  0.090542  0.011983  0.124816 -0.069250
 e/p     0.780559  0.776258  1.000000  0.771022  0.171699  0.204729 -0.003916 -0.042460 -0.184699 -0.389356 -0.354580 -0.033677 -0.067065  0.244407  0.033480
 b/m_FD  0.876660  0.877039  0.771022  1.000000  0.262926  0.249961 -0.020313 -0.035616 -0.059802  0.104776 -0.265900  0.157400 -0.022598  0.169788  0.044291
 ntis    0.224010  0.226061  0.171699  0.262926  1.000000  0.103672 -0.109347 -0.100199 -0.240883  0.065750 -0.136656 -0.419595  0.039599  0.046816 -0.171648
 infl    0.168510  0.177266  0.204729  0.249961  0.1

# Question 3
Apply hierarchical clustering to the same features, as per Chan and Man 2021b (see article on Overview Page). Again, set the random seed to 1. How many clusters did you find?  What are the top 2 clusters? (3 pt)

In [25]:
np.random.seed(1)

## Using clusterKMeansTop_0(3 param)

In [26]:
#---------------------------------------------------
def clusterKMeansTop_00(corr0,maxNumClusters,n_init, n_seed):
    if maxNumClusters==None:maxNumClusters=corr0.shape[1]-1
    corr1,clstrs,silh=clusterKMeansBase_0(corr0,maxNumClusters= \
        min(maxNumClusters,corr0.shape[1]-1),n_init=n_init, seed = n_seed)
    clusterTstats={i:np.mean(silh[clstrs[i]])/ \
        np.std(silh[clstrs[i]]) for i in clstrs.keys()}
    tStatMean=sum(clusterTstats.values())/len(clusterTstats)
    redoClusters=[i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    if len(redoClusters)<=1:
        return corr1,clstrs,silh
    else:
        keysRedo=[j for i in redoClusters for j in clstrs[i]]
        corrTmp=corr0.loc[keysRedo,keysRedo]
        tStatMean=np.mean([clusterTstats[i] for i in redoClusters])
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp)
        #, \
           # maxNumClusters=min(maxNumClusters, \
           # corrTmp.shape[1]-1),n_init=n_init)
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0, \
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, \
            clstrs2)
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys()])
        if newTstatMean<=tStatMean:
            return corr1,clstrs,silh
        else:
            return corrNew,clstrsNew,silhNew

In [27]:
# def clusterKMeansTop_00(corr0,maxNumClusters,n_init):
#            return corrNew,clstrsNew,silhNew

corr3,clstrs3,silh3 = clusterKMeansTop_00(x_train1.corr(), maxNumClusters = 10,
                                        n_init = 7, n_seed = 1)
corr3,clstrs3,silh3

<ipython-input-17-fb9cb799cfd1>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
<ipython-input-8-242de6d6205f>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


(             ltr     corpr      svar       d/e       tms       dfy       dfr       d/p       d/y       e/p    b/m_FD      ntis      infl    tbl_FD    lty_FD
 ltr     1.000000  0.867496  0.124359 -0.009870 -0.012345  0.144381 -0.426798 -0.022039 -0.032731 -0.014742 -0.011490 -0.054186 -0.125912 -0.034961 -0.117420
 corpr   0.867496  1.000000  0.062749  0.051745  0.033279  0.207948  0.079616 -0.033954 -0.047878 -0.066854 -0.036049 -0.064883 -0.146807 -0.069127 -0.115281
 svar    0.124359  0.062749  1.000000  0.246063  0.151751  0.414724 -0.135125 -0.061987 -0.054413 -0.221998 -0.058352 -0.265416 -0.152068 -0.173458 -0.027382
 d/e    -0.009870  0.051745  0.246063  1.000000  0.175410  0.224905  0.113852  0.286564  0.288229 -0.383012  0.108340  0.073208 -0.033817 -0.054944 -0.067177
 tms    -0.012345  0.033279  0.151751  0.175410  1.000000  0.265827  0.085239 -0.241907 -0.247858 -0.348891 -0.274746 -0.156518 -0.172814 -0.417904  0.059073
 dfy     0.144381  0.207948  0.414724  0.224905  0.2

## Using clusterKMeansTop(1 param)



In [28]:
#---------------------------------------------------
def clusterKMeansTop(corr0):
    corr1,clstrs,silh=clusterKMeansBase(corr0)
    clusterTstats={i:np.mean(silh[clstrs[i]])/ \
        np.std(silh[clstrs[i]]) for i in clstrs.keys() if len(clstrs[i])>1}
    tStatMean=sum(np.array(list(clusterTstats.values())))/len(clusterTstats)
    redoClusters=[i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    if len(redoClusters)<=1:
        return corr1,clstrs,silh
    else:
        keysRedo=[j for i in redoClusters for j in clstrs[i]]
        corrTmp=corr0.loc[keysRedo,keysRedo]
        tStatMean=np.mean([clusterTstats[i] for i in redoClusters])
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp)
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0, \
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, \
            clstrs2)
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys() if len(clstrsNew[i])>1])
        if newTstatMean<=tStatMean:
            return corr1,clstrs,silh
        else:
            return corrNew,clstrsNew,silhNew

In [29]:
np.random.seed(1)

In [30]:
corr3a,clstrs3a,silh3a= clusterKMeansTop(x_train1.corr())
corr3a,clstrs3a,silh3a

<ipython-input-8-242de6d6205f>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


(             ltr     corpr      svar       d/e       tms       dfy      ntis      infl       d/p       d/y       e/p    b/m_FD    tbl_FD    lty_FD       dfr
 ltr     1.000000  0.867496  0.124359 -0.009870 -0.012345  0.144381 -0.054186 -0.125912 -0.022039 -0.032731 -0.014742 -0.011490 -0.034961 -0.117420 -0.426798
 corpr   0.867496  1.000000  0.062749  0.051745  0.033279  0.207948 -0.064883 -0.146807 -0.033954 -0.047878 -0.066854 -0.036049 -0.069127 -0.115281  0.079616
 svar    0.124359  0.062749  1.000000  0.246063  0.151751  0.414724 -0.265416 -0.152068 -0.061987 -0.054413 -0.221998 -0.058352 -0.173458 -0.027382 -0.135125
 d/e    -0.009870  0.051745  0.246063  1.000000  0.175410  0.224905  0.073208 -0.033817  0.286564  0.288229 -0.383012  0.108340 -0.054944 -0.067177  0.113852
 tms    -0.012345  0.033279  0.151751  0.175410  1.000000  0.265827 -0.156518 -0.172814 -0.241907 -0.247858 -0.348891 -0.274746 -0.417904  0.059073  0.085239
 dfy     0.144381  0.207948  0.414724  0.224905  0.2

In [31]:
#---------------------------------------------------
def clusterKMeansTop_0(corr0):
    corr1,clstrs,silh=clusterKMeansBase(corr0)
    clusterTstats={i:np.mean(silh[clstrs[i]])/ \
        np.std(silh[clstrs[i]]) for i in clstrs.keys() if len(clstrs[i])>1}
    tStatMean=sum(np.array(list(clusterTstats.values())))/len(clusterTstats)
    redoClusters=[i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    if len(redoClusters)<=1:
        return corr1,clstrs,silh
    else:
        keysRedo=[j for i in redoClusters for j in clstrs[i]]
        corrTmp=corr0.loc[keysRedo,keysRedo]
        tStatMean=np.mean([clusterTstats[i] for i in redoClusters])
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp)
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0, \
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, \
            clstrs2)
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys() if len(clstrsNew[i])>1])
        if newTstatMean<=tStatMean:
            return corr1,clstrs,silh
        else:
            return corrNew,clstrsNew,silhNew

## Using clusterKMeansTop_0(1 param)




In [32]:
np.random.seed(1)

In [33]:
corr3b,clstrs3b,silh3b = clusterKMeansTop(x_train1.corr())
corr3b,clstrs3b,silh3b

<ipython-input-8-242de6d6205f>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


(             ltr     corpr      svar       d/e       tms       dfy      ntis      infl       d/p       d/y       e/p    b/m_FD    tbl_FD    lty_FD       dfr
 ltr     1.000000  0.867496  0.124359 -0.009870 -0.012345  0.144381 -0.054186 -0.125912 -0.022039 -0.032731 -0.014742 -0.011490 -0.034961 -0.117420 -0.426798
 corpr   0.867496  1.000000  0.062749  0.051745  0.033279  0.207948 -0.064883 -0.146807 -0.033954 -0.047878 -0.066854 -0.036049 -0.069127 -0.115281  0.079616
 svar    0.124359  0.062749  1.000000  0.246063  0.151751  0.414724 -0.265416 -0.152068 -0.061987 -0.054413 -0.221998 -0.058352 -0.173458 -0.027382 -0.135125
 d/e    -0.009870  0.051745  0.246063  1.000000  0.175410  0.224905  0.073208 -0.033817  0.286564  0.288229 -0.383012  0.108340 -0.054944 -0.067177  0.113852
 tms    -0.012345  0.033279  0.151751  0.175410  1.000000  0.265827 -0.156518 -0.172814 -0.241907 -0.247858 -0.348891 -0.274746 -0.417904  0.059073  0.085239
 dfy     0.144381  0.207948  0.414724  0.224905  0.2

# Question 4

Set the random seed to 2. Repeat Part 3. How many clusters did you find?  What are the top 2 clusters? (3 pt)

In [34]:
np.random.seed(1)

## Using clusterKMeansTop_0(3 param)

In [35]:
#---------------------------------------------------
def clusterKMeansTop_00(corr0,maxNumClusters,n_init, n_seed):
    if maxNumClusters==None:maxNumClusters=corr0.shape[1]-1
    corr1,clstrs,silh=clusterKMeansBase_0(corr0,maxNumClusters= \
        min(maxNumClusters,corr0.shape[1]-1),n_init=n_init, seed = n_seed)
    clusterTstats={i:np.mean(silh[clstrs[i]])/ \
        np.std(silh[clstrs[i]]) for i in clstrs.keys()}
    tStatMean=sum(clusterTstats.values())/len(clusterTstats)
    redoClusters=[i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    if len(redoClusters)<=1:
        return corr1,clstrs,silh
    else:
        keysRedo=[j for i in redoClusters for j in clstrs[i]]
        corrTmp=corr0.loc[keysRedo,keysRedo]
        tStatMean=np.mean([clusterTstats[i] for i in redoClusters])
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp)
        #, \
           # maxNumClusters=min(maxNumClusters, \
           # corrTmp.shape[1]-1),n_init=n_init)
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0, \
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, \
            clstrs2)
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys()])
        if newTstatMean<=tStatMean:
            return corr1,clstrs,silh
        else:
            return corrNew,clstrsNew,silhNew

In [36]:
# def clusterKMeansTop_00(corr0,maxNumClusters,n_init):
#            return corrNew,clstrsNew,silhNew

corr4,clstrs4,silh4 = clusterKMeansTop_00(x_train1.corr(), maxNumClusters = 10,
                                        n_init = 7, n_seed = 1)
corr4,clstrs4,silh4

<ipython-input-17-fb9cb799cfd1>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
<ipython-input-8-242de6d6205f>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


(             ltr     corpr      svar       d/e       tms       dfy       dfr       d/p       d/y       e/p    b/m_FD      ntis      infl    tbl_FD    lty_FD
 ltr     1.000000  0.867496  0.124359 -0.009870 -0.012345  0.144381 -0.426798 -0.022039 -0.032731 -0.014742 -0.011490 -0.054186 -0.125912 -0.034961 -0.117420
 corpr   0.867496  1.000000  0.062749  0.051745  0.033279  0.207948  0.079616 -0.033954 -0.047878 -0.066854 -0.036049 -0.064883 -0.146807 -0.069127 -0.115281
 svar    0.124359  0.062749  1.000000  0.246063  0.151751  0.414724 -0.135125 -0.061987 -0.054413 -0.221998 -0.058352 -0.265416 -0.152068 -0.173458 -0.027382
 d/e    -0.009870  0.051745  0.246063  1.000000  0.175410  0.224905  0.113852  0.286564  0.288229 -0.383012  0.108340  0.073208 -0.033817 -0.054944 -0.067177
 tms    -0.012345  0.033279  0.151751  0.175410  1.000000  0.265827  0.085239 -0.241907 -0.247858 -0.348891 -0.274746 -0.156518 -0.172814 -0.417904  0.059073
 dfy     0.144381  0.207948  0.414724  0.224905  0.2

## Using clusterKMeansTop(1 param)



In [37]:
#---------------------------------------------------
def clusterKMeansTop(corr0):
    corr1,clstrs,silh=clusterKMeansBase(corr0)
    clusterTstats={i:np.mean(silh[clstrs[i]])/ \
        np.std(silh[clstrs[i]]) for i in clstrs.keys() if len(clstrs[i])>1}
    tStatMean=sum(np.array(list(clusterTstats.values())))/len(clusterTstats)
    redoClusters=[i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    if len(redoClusters)<=1:
        return corr1,clstrs,silh
    else:
        keysRedo=[j for i in redoClusters for j in clstrs[i]]
        corrTmp=corr0.loc[keysRedo,keysRedo]
        tStatMean=np.mean([clusterTstats[i] for i in redoClusters])
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp)
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0, \
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, \
            clstrs2)
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys() if len(clstrsNew[i])>1])
        if newTstatMean<=tStatMean:
            return corr1,clstrs,silh
        else:
            return corrNew,clstrsNew,silhNew

In [38]:
np.random.seed(1)

In [39]:
corr4a,clstrs4a,silh4a= clusterKMeansTop(x_train1.corr())
corr4a,clstrs4a,silh4a

<ipython-input-8-242de6d6205f>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


(             ltr     corpr      svar       d/e       tms       dfy      ntis      infl       d/p       d/y       e/p    b/m_FD    tbl_FD    lty_FD       dfr
 ltr     1.000000  0.867496  0.124359 -0.009870 -0.012345  0.144381 -0.054186 -0.125912 -0.022039 -0.032731 -0.014742 -0.011490 -0.034961 -0.117420 -0.426798
 corpr   0.867496  1.000000  0.062749  0.051745  0.033279  0.207948 -0.064883 -0.146807 -0.033954 -0.047878 -0.066854 -0.036049 -0.069127 -0.115281  0.079616
 svar    0.124359  0.062749  1.000000  0.246063  0.151751  0.414724 -0.265416 -0.152068 -0.061987 -0.054413 -0.221998 -0.058352 -0.173458 -0.027382 -0.135125
 d/e    -0.009870  0.051745  0.246063  1.000000  0.175410  0.224905  0.073208 -0.033817  0.286564  0.288229 -0.383012  0.108340 -0.054944 -0.067177  0.113852
 tms    -0.012345  0.033279  0.151751  0.175410  1.000000  0.265827 -0.156518 -0.172814 -0.241907 -0.247858 -0.348891 -0.274746 -0.417904  0.059073  0.085239
 dfy     0.144381  0.207948  0.414724  0.224905  0.2

In [40]:
#---------------------------------------------------
def clusterKMeansTop_0(corr0):
    corr1,clstrs,silh=clusterKMeansBase(corr0)
    clusterTstats={i:np.mean(silh[clstrs[i]])/ \
        np.std(silh[clstrs[i]]) for i in clstrs.keys() if len(clstrs[i])>1}
    tStatMean=sum(np.array(list(clusterTstats.values())))/len(clusterTstats)
    redoClusters=[i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    if len(redoClusters)<=1:
        return corr1,clstrs,silh
    else:
        keysRedo=[j for i in redoClusters for j in clstrs[i]]
        corrTmp=corr0.loc[keysRedo,keysRedo]
        tStatMean=np.mean([clusterTstats[i] for i in redoClusters])
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp)
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0, \
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, \
            clstrs2)
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys() if len(clstrsNew[i])>1])
        if newTstatMean<=tStatMean:
            return corr1,clstrs,silh
        else:
            return corrNew,clstrsNew,silhNew

## Using clusterKMeansTop_0(1 param)




In [41]:
np.random.seed(4)

In [42]:
corr4b,clstrs4b,silh4b = clusterKMeansTop(x_train1.corr())
corr4b,clstrs4b,silh4b

<ipython-input-8-242de6d6205f>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


(             ltr     corpr      svar       d/e       tms       dfy      ntis      infl       d/p       d/y       e/p    b/m_FD    tbl_FD    lty_FD       dfr
 ltr     1.000000  0.867496  0.124359 -0.009870 -0.012345  0.144381 -0.054186 -0.125912 -0.022039 -0.032731 -0.014742 -0.011490 -0.034961 -0.117420 -0.426798
 corpr   0.867496  1.000000  0.062749  0.051745  0.033279  0.207948 -0.064883 -0.146807 -0.033954 -0.047878 -0.066854 -0.036049 -0.069127 -0.115281  0.079616
 svar    0.124359  0.062749  1.000000  0.246063  0.151751  0.414724 -0.265416 -0.152068 -0.061987 -0.054413 -0.221998 -0.058352 -0.173458 -0.027382 -0.135125
 d/e    -0.009870  0.051745  0.246063  1.000000  0.175410  0.224905  0.073208 -0.033817  0.286564  0.288229 -0.383012  0.108340 -0.054944 -0.067177  0.113852
 tms    -0.012345  0.033279  0.151751  0.175410  1.000000  0.265827 -0.156518 -0.172814 -0.241907 -0.247858 -0.348891 -0.274746 -0.417904  0.059073  0.085239
 dfy     0.144381  0.207948  0.414724  0.224905  0.2